In [1]:
from textblob_de import TextBlobDE
import nbformat
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud 
from functions import *
from gensim.utils import simple_preprocess
from globalvars import *
from IPython.display import display
from IPython.display import Markdown as md
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from pandas_profiling import ProfileReport
import contractions
import gensim
import kaleido
import itertools
import nltk
import numpy as np
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import matplotlib
import matplotlib.pyplot as plt
import plotly.io as pio
import re
import spacy
import string
from functions import *
import csv

# nltk.download('averaged_perceptron_tagger')
# nltk.download('punkt')
# nltk.download('stopwords')

# nlp = spacy.load("de_core_news_lg")
# ## download nlp language package
# #!python -m spacy download de_core_news_lg
# # Load the German model

/Users/dom/opt/anaconda3/envs/nlp_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-08 09:01:08.049322: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
## Import dataframe
filelocation = 'data/DataText'
df = pd.read_feather(filelocation)

In [3]:
df

,participant_id,u_date,year,month,quarter,yearmonth,yearquarter,season,Kommentar,wime_personal,...,ft_zielort,Kommentar_Character,Kommentar_Tokens,Kommentar_Types,Kommentar_TTR,text_preprocessed,tokenized,lemmatized,nouns,nouns_and_adjectives
0,42319,2019-05-28,2019,5,2,2019-05-01,2019/2,spring,Bitte nicht alles elektronifizieren !,NaN,...,Thun,37,5,5,100.000000,bitte elektronifizieren !,"[bitte, elektronifizieren, !]","[bitte, elektronifizier, --]",[elektronifizier],[elektronifizier]
1,42429,2019-02-13,2019,2,1,2019-02-01,2019/1,winter,Bei der Ankunft in Olten um Mitternacht 24 00 ...,77.777778,...,Zofingen,284,50,46,92.000000,ankunft mitternacht 24 00 48 min. gewartet näc...,"[ankunft, mitternacht, 24, 00, 48, min., gewar...","[ankunft, mitternacht, 24, 00, 48, min., warte...","[ankunft, mitternacht, anschluss]","[ankunft, mitternacht, nächster, anschluss]"
2,41926,2019-02-08,2019,2,1,2019-02-01,2019/1,winter,im Zug 6.25 ab St. Gallen sind sehr viel die W...,77.777778,...,Winterthur,65,13,13,100.000000,6.25 wc's gesperrt.,"[6.25, wc's, gesperrt, .]","[6.25, wc's, sperren, --]",[],[]
3,42403,2019-02-07,2019,2,1,2019-02-01,2019/1,winter,BAHNHOF PRATTELN WIEDER BEDIENEN,100.000000,...,Interlaken West,32,4,4,100.000000,bahnhof bedienen,"[bahnhof, bedienen]","[bahnhof, bedienen]",[bahnhof],[bahnhof]
4,42486,2019-02-06,2019,2,1,2019-02-01,2019/1,winter,"Ich kann nicht verstehen, warum mit dem Fahrpl...",NaN,...,Zürich HB,127,18,18,100.000000,"verstehen, fahrplanwechsel langer doppelstockz...","[verstehen, ,, fahrplanwechsel, langer, doppel...","[verstehen, --, fahrplanwechsel, lang, doppels...",[doppelstockzug],[doppelstockzug]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,51510,2019-01-01,2019,1,1,2019-01-01,2019/1,winter,Ich finde auf dem SBB App nirgens eine Option ...,NaN,...,Baar,157,24,24,100.000000,finde sbb app nirgens option nachtzuschlag. me...,"[finde, sbb, app, nirgens, option, nachtzuschl...","[finden, sbb, app, nirgen, option, nachtzuschl...","[app, nirgen, option, meinung]","[app, nirgen, option, meinung]"
386,51505,2019-01-01,2019,1,1,2019-01-01,2019/1,winter,WC's sollten nicht systemmässig geschlossen se...,NaN,...,Brig Bahnhofplatz,124,14,13,92.857143,wc's systemmässig geschlossen sein. stosszeite...,"[wc's, systemmässig, geschlossen, sein, ., sto...","[wc's, systemmässig, schließen, sein, --, stos...","[platzangebot, zugskompositionen]","[grössere, platzangebot, zugskompositionen]"
387,51491,2019-01-01,2019,1,1,2019-01-01,2019/1,winter,mehr platz zur verfügung stellen... jedesmal m...,NaN,...,Luzern,96,14,14,100.000000,platz verfügung stellen... jedesmal stehen. mü...,"[platz, verfügung, stellen, ..., jedesmal, ste...","[platz, verfügung, stellen, --, jedesmal, steh...","[platz, verfügung, vorallem, preis]","[platz, verfügung, vorallem, preis]"
388,51490,2019-01-01,2019,1,1,2019-01-01,2019/1,winter,Die Anschluss-Verbindungen von oder nach Galge...,NaN,...,Landquart,197,29,28,96.551724,anschluss-verbindungen bahnhof siebnen (zu sch...,"[anschluss-verbindungen, bahnhof, siebnen, (, ...","[anschluss-verbindungen, bahnhof, siebn, --, z...","[anschluss-verbindungen, bahnhof, richtung, fa...","[anschluss-verbindungen, bahnhof, richtung, fa..."


https://www.analyticsvidhya.com/blog/2021/12/how-to-extract-key-phrases-using-tfidf-with-python/

In [3]:
# df.u_ticket
df.columns

Index(['participant_id', 'u_date', 'year', 'month', 'quarter', 'yearmonth',
       'yearquarter', 'season', 'Kommentar', 'wime_personal', 'wime_komfort',
       'wime_sauberkeit', 'wime_puenktlich', 'wime_platzangebot',
       'wime_gesamtzuf', 'wime_preis_leistung', 'wime_fahrplan',
       'wime_oes_fahrt', 'S_sprache', 'S_alter', 'S_sex', 'S_wohnsitz',
       'u_klassencode', 'S_AB3_HTA', 'R_anschluss', 'R_stoerung',
       'device_type', 'dispcode', 'u_ticket', 'u_fahrausweis', 'u_preis',
       'R_zweck', 'ft_abfahrt', 'ft_ankunft', 'ft_startort_uic', 'ft_tu',
       'ft_vm', 'ft_vm_kurz', 'ft_zielort_uic', 'fg_abfahrt', 'fg_ankunft',
       'fg_startort_uic', 'fg_zielort_uic', 'fg_startort', 'fg_zielort',
       'ft_startort', 'ft_zielort', 'Kommentar_Character', 'Kommentar_Tokens',
       'Kommentar_Types', 'Kommentar_TTR', 'text_preprocessed', 'tokenized',
       'lemmatized', 'lemmatized_no_loc', 'nouns'],
      dtype='object')

In [14]:
corpus = join_list_of_list(df['nouns'])

# Comparing two corpus
corpus_1 = df[df['S_sex'] == 'weiblich']
corpus_1 =join_list_of_list(corpus_1['nouns'])

corpus_2 = df[df['S_sex'] == 'männlich']
corpus_2 =join_list_of_list(corpus_2['nouns'])

In [15]:
vectorizer_params = {'ngram_range': (2,3), 'max_df': 0.99,'min_df':1}
get_tfidf(corpus=corpus,vectorizer_params=vectorizer_params,n_keywords=20)

: 

: 

In [6]:
vectorizer_params = {'ngram_range': (1,2), 'max_df': 0.1,'min_df':10}
compare_tfidf(corpus_2,corpus_1,vectorizer_params=vectorizer_params,n_keywords=20)

züge            224.122495
leute           179.358784
platz           167.735646
menschen        167.150151
kinderwagen     157.271480
strecke         148.612638
bahnhof         143.674114
sitzplätze      128.435955
zügen           125.019717
toiletten       121.059645
maske           120.052572
sauberkeit      116.224008
verbindungen    114.271775
gepäck          112.566586
fahrt           103.965967
verspätung       99.424391
verbindung       99.239835
klasse           93.173354
koffer           92.472989
masken           83.961235
dtype: float64

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

def extract_keywords(texts, num_keywords=10):
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(texts)

    feature_names = tfidf.get_feature_names_out()
    dense = tfidf_matrix.todense()
    denselist = dense.tolist()
    texts_keywords = [
        [(feature_names[index], scores) for index, scores in zip(text.indices, text.data) if scores > 0]
        for text in denselist
    ]

    keywords = {}
    for text_keywords in texts_keywords:
        for word, score in text_keywords:
            if word in keywords:
                keywords[word] += score
            else:
                keywords[word] = score

    keywords = sorted(keywords.items(), key=lambda x: x[1], reverse=True)
    keywords = keywords[:num_keywords]
    return keywords


In [8]:
extract_keywords(df['Kommentar'], num_keywords=10)

: 

: 

In [26]:
type(df['nouns'])

pandas.core.series.Series

In [12]:
trending_df = calculate_trending_keywords_tfidf(df=df,group_column='season',text_column='nouns',min_df=0.1,max_df=0.9)

ValueError: After pruning, no terms remain. Try a lower min_df or a higher max_df.

In [10]:
trending_df

,Keyword,Score,Group,Rank,Count,Count Relative
0,zug,0.557148,2022-12-01,1,0.557,0.273441
1,klasse,0.241484,2022-12-01,2,0.241,0.118311
2,sbb,0.238327,2022-12-01,3,0.238,0.116838
3,zürich,0.190977,2022-12-01,4,0.191,0.093765
4,bern,0.164146,2022-12-01,5,0.164,0.080511
...,...,...,...,...,...,...
5,züge,0.138766,2019-01-01,6,0.139,0.069292
6,verbindung,0.126659,2019-01-01,7,0.127,0.063310
7,bahnhof,0.120140,2019-01-01,8,0.120,0.059821
8,ist,0.120140,2019-01-01,9,0.120,0.059821


In [11]:
#trending_df
convert_to_wide_format(trending_df)

,Keyword 1,Keyword 2,Keyword 3,Keyword 4,Keyword 5,Keyword 6,Keyword 7,Keyword 8,Keyword 9,Keyword 10
2022-12-01,zug,klasse,sbb,zürich,bern,züge,bahnhof,ist,platz,app
2022-11-01,zug,sbb,zürich,klasse,züge,bahnhof,app,bern,wagen,fahrt
2022-10-01,zug,sbb,zürich,app,klasse,bahnhof,züge,platz,wagen,bern
2022-09-01,zug,zürich,sbb,klasse,platz,bahnhof,züge,bern,werden,wagen
2022-08-01,zug,sbb,zürich,klasse,bern,züge,wagen,platz,app,bahnhof
2022-07-01,zug,sbb,zürich,platz,klasse,züge,wagen,app,bern,bahnhof
2022-06-01,zug,sbb,züge,zürich,klasse,app,platz,bern,wagen,bahnhof
2022-05-01,zug,sbb,klasse,zürich,züge,bahnhof,platz,ist,wagen,app
2022-04-01,zug,zürich,sbb,klasse,bahnhof,züge,bern,strecke,ist,wagen
2022-03-01,zug,sbb,klasse,zürich,bahnhof,strecke,platz,werden,züge,wagen


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

def get_tfidf_vectors(texts, ngram_range=(1,2)):
    """
    Returns a list of dictionaries containing the tf-idf scores for each term in each text.
    """
    vectorizer = TfidfVectorizer(use_idf=True, max_df=0.20, min_df=1, ngram_range=ngram_range)
    vectors = vectorizer.fit_transform(texts)
    dict_of_tokens = {i[1]:i[0] for i in vectorizer.vocabulary_.items()}
    tfidf_vectors = []
    for row in vectors:
        tfidf_vectors.append({dict_of_tokens[column]:value for (column,value) in zip(row.indices,row.data)})
    return tfidf_vectors

from sklearn.feature_extraction.text import TfidfVectorizer

def extract_top_keywords(corpus, N):
    vectorizer = TfidfVectorizer(use_idf=True, max_df=0.20,min_df=1, ngram_range=(1,2))
    vectors = vectorizer.fit_transform(corpus)
    scores = vectors.toarray().sum(axis=0)
    vocab = vectorizer.vocabulary_
    keywords = [(term, scores[index]) for term, index in vocab.items()]
    keywords = sorted(keywords, key=lambda x: x[1], reverse=True)
    top_keywords = keywords[:N]
    return top_keywords


In [5]:
df_filtered = df[df['season'] == 'summer']

In [6]:
texts = df_filtered['nouns_location_removed_str']
tfidf_vectors = get_tfidf_vectors(texts, ngram_range=(1,3))

KeyError: 'nouns_location_removed_str'

In [ ]:
top_keywords = extract_top_keywords(texts, 20)

In [ ]:
top_keywords

[('sbb', 270.2272351753645),
 ('verbindung', 218.18687857058507),
 ('preis', 212.1203815765708),
 ('platz', 205.7054484544187),
 ('app', 199.2926111995926),
 ('verspätung', 163.32726414951395),
 ('züge', 158.22297928774728),
 ('strecke', 153.4774236069616),
 ('bahnhof', 144.32522531789604),
 ('maskenpflicht', 143.83418725673877),
 ('klimaanlage', 137.412951098461),
 ('bern', 130.91165784860297),
 ('züg', 129.20285116314722),
 ('zeit', 114.01342914811875),
 ('ticket', 110.85303105908879),
 ('bus', 109.689467968056),
 ('reise', 106.43615089106923),
 ('leute', 98.70640982763535),
 ('maske', 96.70920402192174),
 ('velo', 96.6874780255895)]

### RAKE

Cloning into 'RAKE-tutorial'...
remote: Enumerating objects: 303, done.
remote: Total 303 (delta 0), reused 0 (delta 0), pack-reused 303
Receiving objects: 100% (303/303), 4.31 MiB | 1.09 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [7]:
#!pip install rake-nltk
from rake_nltk import Rake
import operator

Version 1

In [7]:
def extract_top_group_keywords(df, group_col, text_col, n=10, group_values=None, tfidf_params=None):
    """
    Extracts the top n keywords for a given group or groups in a dataframe.
    
    Parameters:
    - df (pd.DataFrame): The dataframe containing the data
    - group_col (str): The name of the column in the dataframe that holds the group information
    - text_col (str): The name of the column in the dataframe that holds the text information
    - n (int, optional): The number of top keywords to extract. Default is 10
    - group_values (tuple, optional): The values of the group or groups to extract the top keywords for.
                                      Default is None, which means all groups.
    - tfidf_params (dict, optional): Additional parameters to pass to the TfidfVectorizer class.
                                     Default is None, which means no additional parameters.
    
    Returns:
    - list: A list of the top n keywords
    
    """
    if group_values:
        df = df[df[group_col].isin(group_values)]

    corpus = df[text_col].apply(lambda x: " ".join(x)).tolist()
    
    vectorizer = TfidfVectorizer(**(tfidf_params if tfidf_params else {}))
    X = vectorizer.fit_transform(corpus)
    feature_names = vectorizer.get_feature_names_out()
    tfidf_scores = pd.DataFrame(X.toarray(), columns=feature_names)

    top_keywords = []
    for i, row in tfidf_scores.iterrows():
        sorted_words = row.sort_values(ascending=False).index.tolist()
        top_keywords += sorted_words[:n]
        
    return top_keywords[:n]




Version 2

In [8]:
def extract_top_group_keywords(df, group_col, text_col, n=10, group_values=None, tfidf_params=None):
    """
    Extract the top `n` keywords that are more present in a subgroup compared to the rest of the corpus (all other groups).
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing the data.
    group_col : str
        Column in `df` that contains the subgroup information.
    text_col : str
        Column in `df` that contains the text data.
    n : int, optional (default=10)
        Number of keywords to extract for each subgroup.
    group_values : list, optional (default=None)
        List of values in `group_col` to include in the analysis. If None, include all unique values.
    tfidf_params : dict, optional (default=None)
        Dictionary of parameters to pass to `TfidfVectorizer`.
        
    Returns
    -------
    group_keywords : dict
        Dictionary where the keys are the subgroup names and the values are lists of the top `n` keywords
        that are more present in that subgroup compared to the rest of the corpus (all other groups).
    """
    if group_values:
        df = df[df[group_col].isin(group_values)]

    vectorizer = TfidfVectorizer(**(tfidf_params if tfidf_params else {}))
    X = vectorizer.fit_transform(df[text_col].apply(lambda x: " ".join(x)))
    feature_names = vectorizer.get_feature_names_out()
    tfidf_scores = pd.DataFrame(X.toarray(), columns=feature_names)
    
    group_keywords = {}
    for group in df[group_col].unique():
        group_df = df[df[group_col] == group]
        group_tfidf_scores = tfidf_scores.iloc[group_df.index, :]
        
        group_mean = group_tfidf_scores.mean()
        group_mean_sorted = group_mean.sort_values(ascending=False)
        
        all_other_groups_df = df[df[group_col] != group]
        all_other_groups_tfidf_scores = tfidf_scores.iloc[all_other_groups_df.index, :]
        all_other_groups_mean = all_other_groups_tfidf_scores.mean()
        
        more_present_keywords = []
        for i, (word, score) in enumerate(group_mean_sorted.iteritems()):
            if score > all_other_groups_mean[word]:
                more_present_keywords.append(word)
            if i >= n:
                break
        
        group_keywords[group] = more_present_keywords
        
    return group_keywords



Version 3

In [9]:
from typing import Optional

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

def extract_top_group_keywords(df, group_col, text_col, n=10, group_values=None, tfidf_params=None, sort=False, compare_with_all=False):
    """
    Extract the top `n` keywords that are more present in a subgroup compared to the rest of the corpus (all other groups).
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing the data.
    group_col : str
        Column in `df` that contains the subgroup information.
    text_col : str
        Column in `df` that contains the text data.
    n : int, optional (default=10)
        Number of keywords to extract for each subgroup.
    group_values : list, optional (default=None)
        List of values in `group_col` to include in the analysis. If None, include all unique values.
    tfidf_params : dict, optional (default=None)
        Dictionary of parameters to pass to `TfidfVectorizer`.
    sort : bool, optional (default=False)
        Sort the keywords by their scores in each group.
    compare_with_overall : bool, optional (default=False)
        Compare the keyword scores of each group with the overall mean of the corpus instead of with the scores of all other groups.
        
    Returns
    -------
    group_keywords : dict
        Dictionary where the keys are the subgroup names and the values are lists of the keyword scores in each subgroup.
    """

    if group_values:
        df = df[df[group_col].isin(group_values)]

    vectorizer = TfidfVectorizer(**(tfidf_params if tfidf_params else {}))
    X = vectorizer.fit_transform(df[text_col].apply(lambda x: " ".join(x)))
    feature_names = vectorizer.get_feature_names_out()
    tfidf_scores = pd.DataFrame(X.toarray(), columns=feature_names)

    group_keywords = {}
    all_group_mean = tfidf_scores.mean()
    for group in df[group_col].unique():
        group_df = df[df[group_col] == group]
        group_tfidf_scores = tfidf_scores.iloc[group_df.index, :]
        
        group_mean = group_tfidf_scores.mean()
        if sort:
            group_mean_sorted = group_mean.sort_values(ascending=False)
        else:
            group_mean_sorted = group_mean
        
        if compare_with_all:
            all_other_groups_mean = all_group_mean
        else:
            all_other_groups_df = df[df[group_col] != group]
            all_other_groups_tfidf_scores = tfidf_scores.iloc[all_other_groups_df.index, :]
            all_other_groups_mean = all_other_groups_tfidf_scores.mean()
        
        more_present_keywords = []
        for i, (word, score) in enumerate(group_mean_sorted.iteritems()):
            if score > all_other_groups_mean[word]:
                more_present_keywords.append((word, score))
            if i >= n:
                break
        
        group_keywords[group] = more_present_keywords
    
        
    return group_keywords

Version 4

In [10]:
def extract_group_representative_keywords(df, group_col, text_col, n=10, group_values=None, tfidf_params=None, sort=False):
    if group_values:
        df = df[df[group_col].isin(group_values)]

    vectorizer = TfidfVectorizer(**(tfidf_params if tfidf_params else {}))
    X = vectorizer.fit_transform(df[text_col].apply(lambda x: " ".join(x)))
    feature_names = vectorizer.get_feature_names_out()
    tfidf_scores = pd.DataFrame(X.toarray(), columns=feature_names)

    group_representative_keywords = {}
    for group in df[group_col].unique():
        group_df = df[df[group_col] == group]
        group_tfidf_scores = tfidf_scores.iloc[group_df.index, :]
        group_mean = group_tfidf_scores.mean()

        if sort:
            group_mean_sorted = group_mean.sort_values(ascending=False)
        else:
            group_mean_sorted = group_mean

        representative_keywords = []
        for i, (word, score) in enumerate(group_mean_sorted.iteritems()):
            representative_keywords.append((word, score))
            if i >= n:
                break
        
        group_representative_keywords[group] = representative_keywords
    
    return group_representative_keywords

Version 5

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

def category_based_tfidf(df, column, category_column, tfidf_params=None):
    # create a dict with the frequency of each word in each category
    freq = defaultdict(lambda: defaultdict(int))
    for i, row in df.iterrows():
        category = row[category_column]
        if isinstance(row[column], list):
            for sublist in row[column]:
                if isinstance(sublist, np.ndarray):
                    sublist = sublist.tolist()
                for word in str(sublist).split():
                    freq[category][word] += 1
        else:
            for word in str(row[column]).split():
                freq[category][word] += 1
    # convert the dict to a dataframe for further processing
    freq_df = pd.DataFrame(freq).transpose().fillna(0)
    # convert the frequency data into a list of strings
    strings = []
    for i, row in freq_df.iterrows():
        strings.append(" ".join([f"{k}:{v}" for k,v in row.to_dict().items()]))
    # calculate tf-idf for each word in each category
    if tfidf_params is None:
        vectorizer = TfidfVectorizer(use_idf=True)
    else:
        vectorizer = TfidfVectorizer(use_idf=True, **tfidf_params)
    tfidf = vectorizer.fit_transform(strings)
    # return the tf-idf score in a dataframe
    return pd.DataFrame(tfidf.todense(), index=freq_df.index, columns=vectorizer.get_feature_names_out()).transpose()





In [5]:
df_test = df
tfidf_params={"max_df": 0.9, "min_df": 0.001}

In [6]:
results = category_based_tfidf(df_test, 'nouns', 'season',tfidf_params=None)

In [8]:
df

,participant_id,u_date,year,month,quarter,yearmonth,yearquarter,season,Kommentar,wime_personal,...,ft_zielort,Kommentar_Character,Kommentar_Tokens,Kommentar_Types,Kommentar_TTR,text_preprocessed,tokenized,lemmatized,lemmatized_no_loc,nouns
0,612374,2022-12-31,2022,12,4,2022-12-01,2022/4,winter,Häufigere Verbindungen zw. Bern-Luzern.,75.0,...,Luzern,39,4,4,100.000000,häufigere verbindungen zw. bern-luzern.,"[bern, luzern, .]",[--],[--],[]
1,604479,2022-12-31,2022,12,4,2022-12-01,2022/4,winter,"Als Student ist das GA Preiswert, aber mehr wü...",NaN,...,BÜREN AN DER AARE,180,32,28,87.500000,"student ga preiswert, gerne bezahlen. 1 monats...",[],[],[],[]
2,604528,2022-12-31,2022,12,4,2022-12-01,2022/4,winter,Weniger 1. Klasswagons und mehr 2. Klassenwago...,100.0,...,Bern,140,23,22,95.652174,1. klasswagons 2. klassenwagons. genau ir16 (b...,"[(, bern, aarau, aarau, bern, )]","[--, --]","[--, --]",[]
3,604518,2022-12-31,2022,12,4,2022-12-01,2022/4,winter,Immer wieder grosses Problem ist kaputt gescho...,NaN,...,ZÜRICH HB,75,12,12,100.000000,problem kaputt geschossen wc. züge (s9).,[],[],[],[]
4,604504,2022-12-31,2022,12,4,2022-12-01,2022/4,winter,Ich bin Pendlerin ( und glückliche Inhaberin e...,NaN,...,Chur,331,52,46,88.461538,pendlerin ( glückliche inhaberin gas) pendle f...,"[zweitklass, wagons]",[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64340,51540,2019-01-01,2019,1,1,2019-01-01,2019/1,winter,Dieser Zug war ab Zürich HB ein Nachtzug. Die ...,NaN,...,Stettbach,201,34,32,94.117647,zug zürich hb nachtzug. app nachtzuschlag verr...,[],[],[],[]
64341,51531,2019-01-01,2019,1,1,2019-01-01,2019/1,winter,Übersichtlichkeit der App.Es könnte besser ang...,100.0,...,Basel SBB,185,23,23,100.000000,"übersichtlichkeit app.es angegeben werden, tic...",[],[],[],[]
64342,51529,2019-01-01,2019,1,1,2019-01-01,2019/1,winter,Bei der Rückfahrt hatten wir Sitzplätze aber b...,100.0,...,Thun,450,73,60,82.191781,rückfahrt sitzplätze anreise thun interlaken s...,"[sind:, )]",[--],[--],[]
64343,51527,2019-01-01,2019,1,1,2019-01-01,2019/1,winter,Mehr Spielwagen auf unterschiedlichen Strecken...,100.0,...,Chur,59,7,7,100.000000,spielwagen unterschiedlichen strecken wären su...,[],[],[],[]


In [32]:
results

,winter,autumn,summer,spring
00,0.027633,0.027223,0.02717,0.028286
000,0.001454,0.001433,0.00143,0.001489
00fr,0.005817,0.005731,0.00572,0.005955
00uhr,0.017452,0.017194,0.01716,0.017865
01,0.014544,0.014328,0.01430,0.014887
...,...,...,...,...
übrigen,0.001454,0.001433,0.00143,0.001489
übrtprüft,0.001454,0.001433,0.00143,0.001489
übung,0.004363,0.004298,0.00429,0.004466
übungen,0.001454,0.001433,0.00143,0.001489


In [28]:
results['summer'].sort_values(ascending=False)

95     0.242775
92     0.242775
75     0.242775
71     0.242775
159    0.190177
         ...   
273    0.000000
269    0.000000
268    0.000000
263    0.000000
101    0.000000
Name: summer, Length: 178, dtype: float64

In [24]:
# Extract the top 3 keywords only for the Female group
group_values = list(df.S_sex.unique())
tfidf_params = {
    #'min_df': 0.001,
    #'max_df': 0.5,
    'ngram_range':(1,1)
}

In [25]:
top_keywords_new = extract_top_group_keywords(df_test, 'S_sex', 'nouns', n=10, group_values=group_values, tfidf_params=tfidf_params, sort=True, compare_with_all=True)

/var/folders/0q/zp8d2_c16z32wsh2hdjb9fch0000gn/T/ipykernel_34856/1578814117.py:63: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, (word, score) in enumerate(group_mean_sorted.iteritems()):


In [26]:
top_keywords = extract_group_representative_keywords(df_test, 'yearquarter', 'nouns', n=10, group_values=group_values, tfidf_params=tfidf_params, sort=True)

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [27]:
def to_df(group_keywords, n=10):
    groups = list(group_keywords.keys())
    keywords = []
    for i in range(n):
        keyword_column = []
        for group in groups:
            try:
                keyword_column.append(group_keywords[group][i])
            except IndexError:
                keyword_column.append(None)
        keywords.append(keyword_column)

    df = pd.DataFrame({'Group': groups})
    for i, keyword_column in enumerate(keywords):
        df[f'Keyword_{i+1}'] = keyword_column
    
    return df

In [28]:
to_df(top_keywords_new,n=10)

,Group,Keyword_1,Keyword_2,Keyword_3,Keyword_4,Keyword_5,Keyword_6,Keyword_7,Keyword_8,Keyword_9,Keyword_10
0,weiblich,"(zug, 0.07613526190015996)","(züge, 0.03198508972572204)","(bahnhof, 0.020381162425043457)","(verspätung, 0.015890812059224952)","(app, 0.015766849563078514)","(fahrt, 0.015387310939313703)","(strecke, 0.013799337069380286)","(verbindung, 0.013339009915221332)",None,None
1,männlich,"(klasse, 0.03038435640214663)","(platz, 0.021654728275532373)","(minuten, 0.010923924368636626)",None,None,None,None,None,None,None
2,divers,"(fragen, 0.0517727650996974)","(bahnhof, 0.050282481323229905)","(verbindungen, 0.0473328137881326)","(pünktlichkeit, 0.0469683049856828)","(platz, 0.041575886571870314)","(krise, 0.03125)","(wlan, 0.03125)","(klasse, 0.03125)","(kinderwagen, 0.03125)","(takt, 0.03125)"
